### This notebook will use Service Principal 
### to connect to Storage account
### and create a mount point (e.g. /mnt/raw)

In [0]:
client_id = dbutils.secrets.get(scope="db-ma", key="ma-client-id")
tenant_id = dbutils.secrets.get(scope="db-ma", key="ma-tenant-id")
client_secret = dbutils.secrets.get(scope="db-ma", key="ma-secret-id")

This command is used to connect to storage account using service principal

In [0]:
spark.conf.set("fs.azure.account.auth.type.samedallionarchitecture.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.samedallionarchitecture.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.samedallionarchitecture.dfs.core.windows.net", client_id)
spark.conf.set("fs.azure.account.oauth2.client.secret.samedallionarchitecture.dfs.core.windows.net", client_secret)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.samedallionarchitecture.dfs.core.windows.net", f"https://login.microsoftonline.com/{tenant_id}/oauth2/token")

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": client_id,
          "fs.azure.account.oauth2.client.secret": client_secret,
          "fs.azure.account.oauth2.client.endpoint": f"https://login.microsoftonline.com/{tenant_id}/oauth2/token"}

In [0]:
mntlist = []
for i in dbutils.fs.mounts():
    if i.mountPoint.startswith("/mnt/"):
        mntlist.append(i.mountPoint[5:])


In [0]:
lst = ["raw","processed","practice"]

for c in lst:
    if c not in mntlist:
        dbutils.fs.mount(
            source = f"abfss://{c}@samedallionarchitecture.dfs.core.windows.net",
            mount_point = f"/mnt/{c}",
            extra_configs = configs)

In [0]:
dbutils.fs.mounts()